## CO2 flux analysis

This script gets the raw inversion data and calculates 

### Methods:

1. Load inversion CO2 data and preprocess:
    - select roi
    - adjust units
    - set nodata
    - ...
3. Create a grouping coordinate that divides growing and non-growing seasons.   
    - This can be done manually, but consider fitting a harmonic to divide seasons.  
4. For each year and season, calculate the CO2 flux sum.
5. For each year, get the difference between growing and non-growing season.
    - This is the CO2 flux amplitude that can then be related to atmospheric CO2 and driving variables. 


In [ ]:
# Get all required packages

#%%
%load_ext autoreload
%autoreload 2

# Packages
import os

# Local modules
from mod_ra1 import *

### Options

In [1]:
do_plotting = False

# Choose the list of inversion data version to analyse. Later (shorter) time series are driven by data from more stations.
inversion = {'s85':1985, 's99':1999, 's06':2006, 's10':2010}  # Inversion starting year. Options: s85, s99, s06, s10

# Choose last year for analysis period
stop = 2021


### General Settings

In [ ]:
# Directories
input_dir = '../data_input/'
output_dir = '../data_output/'

# Default data properties
epsg_crs = 4326

# Region files available
conts_file = os.path.join(input_dir, 'continents.geojson')
north50_file = os.path.join(input_dir, 'north50.geojson')
north30_file = os.path.join(input_dir, 'north30.geojson')
north0_file = os.path.join(input_dir, 'north0.geojson')
testrect_file = os.path.join(input_dir, 'testrect.geojson')

# Set the region to use for the analysis
roi_file = conts_file


In [ ]:
# Loop over inversion versions and analysis periods

for inv in inversion:
    inv_start = inversion[inv]
    for start in inversion.values():
            if(inv_start >= start):
                continue

            # Analysis ----

            # Chose the inversion data version. Later (shorter) time series are driven by data from more stations.
            co2inv_file = '/Users/moyanofe/BigData/GeoSpatial/Carboscope/Inversions/' + inv + 'oc_v2022_daily.nc'

            # Call function for inversion CO2 preprocessing
            co2fluxProc = procInvCO2(invco2_file=co2inv_file, roi_file=roi_file)

            # Call function to get the yearly seasonal CO2 flux amplitude
            co2fluxAmp = get_CO2fluxSeasAmp(co2fluxProc)

            # Call function to calculate temporal trends, mean, variance, etc in seasonal amplitudes
            co2fluxAmpStats = get_co2fluxampstats(co2fluxAmp)


            # Save to file ----

            # Saving the amp and amp-trend results to disk
            ident = inv + '_' + str(start)+'-'+str(stop)
            file_out_amp = os.path.join(output_dir, 'co2fluxAmp_'+ident+'.nc')
            co2fluxAmp.to_netcdf(file_out_amp)

            file_out_stats = os.path.join(output_dir, 'co2fluxAmpStats_'+ident+'.nc')
            co2fluxAmpStats.to_netcdf(file_out_stats)

            # Save the trend data to GeoTiff (useful if reading into Earth Engine is required)
            # Saving the database (not as an array) keeps the band name info. (But seems like Earth Engine does not import band names)
            # file_out = os.path.join(output_dir, 'co2fluxAmpStats_Inv' + inversion + '.tif')
            # co2fluxAmpStats.rio.to_raster(file_out)

            # Save the co2 flux seasonal difference data to GeoTiff
            # file_out = os.path.join(output_dir, 'co2fluxAmp_Inv' + inversion + '.tif')
            # da_fluxamp = co2fluxAmp['co2flux_yearlyamp']
            # da_fluxamp.rio.to_raster(file_out)


            # Plotting ----

            if(do_plotting):
            
                # Plot the first band of co2 flux rates
                plotOrthographic(co2fluxProc=co2fluxProc)

                # Plotting of seasonal amplitude time series
                plotRegMeans(co2fluxAmp=co2fluxAmp, lats=([30, 50], [50, 90]))

                # Map plot of flux trends in the northern hemisphere
                plotLambert(co2fluxAmpStats.slope)

                # Plot values with significance filter
                plotOrthoSig(co2fluxAmpStats.slope, co2fluxAmpStats.p_value)